# 출처. [캐글타이타닉](https://www.kaggle.com/munmun2004/titanic-for-begginers/comments#871092)

In [1]:
#!kaggle competitions download -c titanic

In [2]:
!ls

gender_submission.csv  sample_data  test.csv  train.csv


# 데이터 load

In [3]:
import pandas as pd
import numpy as np

In [4]:
test_df = pd.read_csv('./test.csv')
train_df = pd.read_csv('./train.csv')
submission_df = pd.read_csv('gender_submission.csv')
print ( test_df.shape, train_df.shape, submission_df.shape)

(418, 11) (891, 12) (418, 2)


In [5]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
submission_df.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# 데이터 분석
test로 submission 의 값을 나오게 하는게 목표인듯 함. train에서는 Survived 를 빼서 target으로 만들어 줘야할거 같고, submission 에서는 불필요한 PassengerId 를 지우고 0,1 을 기반으로 원핫 인코딩을 해줘야 할 것 같음.

In [8]:
train_target = train_df['Survived']
print( train_target.shape, train_target.head())

(891,) 0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [9]:
submission_df = submission_df['Survived']
print(  submission_df.shape, submission_df.head())

(418,) 0    0
1    1
2    0
3    0
4    1
Name: Survived, dtype: int64


In [10]:
from tensorflow import keras

train_y = keras.utils.to_categorical(train_target, 2)
submission_y = keras.utils.to_categorical(submission_df, 2)
print(train_y.shape, submission_y.shape)

(891, 2) (418, 2)


In [11]:
train_x = train_df.drop(['PassengerId','Survived', 'Name', 'Ticket', 'Embarked', 'Cabin'], axis=1, inplace=False)
#train_x = train_x.to_numpy()
test_x = test_df.drop([ 'PassengerId','Name', 'Ticket', 'Embarked', 'Cabin'], axis=1, inplace=False)
#test_x = test_x.to_numpy()
print (train_x, test_x, train_x.shape)

     Pclass     Sex   Age  SibSp  Parch     Fare
0         3    male  22.0      1      0   7.2500
1         1  female  38.0      1      0  71.2833
2         3  female  26.0      0      0   7.9250
3         1  female  35.0      1      0  53.1000
4         3    male  35.0      0      0   8.0500
..      ...     ...   ...    ...    ...      ...
886       2    male  27.0      0      0  13.0000
887       1  female  19.0      0      0  30.0000
888       3  female   NaN      1      2  23.4500
889       1    male  26.0      0      0  30.0000
890       3    male  32.0      0      0   7.7500

[891 rows x 6 columns]      Pclass     Sex   Age  SibSp  Parch      Fare
0         3    male  34.5      0      0    7.8292
1         3  female  47.0      1      0    7.0000
2         2    male  62.0      0      0    9.6875
3         3    male  27.0      0      0    8.6625
4         3  female  22.0      1      1   12.2875
..      ...     ...   ...    ...    ...       ...
413       3    male   NaN      0      

In [12]:
#train_x [train_x.Sex == 'male']['Sex'] = 1
change_dict = {'male' : 1., 'female' : 0. }
train_x = train_x.replace({'Sex' : change_dict})
train_x.fillna(0, inplace=True)
test_x = test_x.replace({'Sex' : change_dict})
test_x.fillna(0, inplace=True)


In [13]:
print (test_x)

     Pclass  Sex   Age  SibSp  Parch      Fare
0         3  1.0  34.5      0      0    7.8292
1         3  0.0  47.0      1      0    7.0000
2         2  1.0  62.0      0      0    9.6875
3         3  1.0  27.0      0      0    8.6625
4         3  0.0  22.0      1      1   12.2875
..      ...  ...   ...    ...    ...       ...
413       3  1.0   0.0      0      0    8.0500
414       1  0.0  39.0      0      0  108.9000
415       3  1.0  38.5      0      0    7.2500
416       3  1.0   0.0      0      0    8.0500
417       3  1.0   0.0      1      1   22.3583

[418 rows x 6 columns]


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(8, activation='sigmoid', input_shape=(6,)))
model.add(Dense(4, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))

model.add(Dense(2, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 56        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 25        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 12        
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


In [15]:

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

In [16]:
model.fit(train_x.astype(float), train_y,
          batch_size=10,
          epochs=100,
          verbose=1
)

Epoch 1/100
90/90 [==============================] - 1s 1ms/step - loss: 0.6711 - accuracy: 0.6162
Epoch 2/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6675 - accuracy: 0.6162
Epoch 3/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6664 - accuracy: 0.6162
Epoch 4/100
90/90 [==============================] - 0s 1ms/step - loss: 0.6660 - accuracy: 0.6162
Epoch 5/100
90/90 [==============================] - 0s 1ms/step - loss: 0.6649 - accuracy: 0.6162
Epoch 6/100
90/90 [==============================] - 0s 1ms/step - loss: 0.6645 - accuracy: 0.6162
Epoch 7/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6632 - accuracy: 0.6162
Epoch 8/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6622 - accuracy: 0.6162
Epoch 9/100
90/90 [==============================] - 0s 1ms/step - loss: 0.6599 - accuracy: 0.6162
Epoch 10/100
90/90 [==============================] - 0s 1ms/step - loss: 0.6581 - accuracy: 0.6162
Epoch 11/

In [17]:
score = model.evaluate(test_x.astype(float), submission_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.2657574713230133
Test accuracy: 0.9473684430122375
